In [18]:
# Calculates success rates for all methods aside from BMA

# Generates a specified number of test sets each with 12 participants with uncertainty and error determined from specified 
# distribution, and determines the proportion of the labs which fail each test.

# Can update the participants function to generate success rates for different combinations of biased and unbiased
# participants. See txt file in same folder as this to find other versions of the 'participants' function.

code_version = 1

In [2]:
# Run methods notebook which contains functions for each of the methods used
%run "Methods-success-new-GTC.ipynb" -G

In [3]:
# Two biased - can set to be ++ or +- by adjusting the sign.

def participants(N,f,g,d):
    # Generate a participant set, run the methods and return the required variables to determine success
    # Takes in: N = number of participants
    #           f = uncertainty of the biased lab
    #           g = size of bias
    #           d = degrees of freedom of Chi Square distribution used to generated uncertainties
    # Returns:  y = uncertain array containing N participants, the first of which has a bias defined by f,g
    
    unc = np.array([np.random.chisquare(d)/d for i in range(0,N-2)])
    unc = np.insert(unc, 0, f) # Set the first lab to have f=1
    unc = np.insert(unc, 0, f) # Set the first lab to have f=1

    # Generate errors from a Normal (0,unc) distribution
    err = np.array([random.normalvariate(0,unc[i]) for i in range(2,N)])
    err = np.insert(err, 0, [random.normalvariate(0,unc[1])+g]) # Set the first lab to have a bias of 6*unc
    err = np.insert(err, 0, [random.normalvariate(0,unc[0])+g]) # Set the first lab to have a bias of 6*unc

    # Put 12 participants into an uncertain number array
    y = la.uarray([ureal(err[i],unc[i]) for i in range(0,N)])
    
    n_biased = 2
    
    return(y, n_biased)

In [4]:
def run_methods(y,N,d,u_d):
    # Run the data through all the different methods
    # Takes in: y = an uncertain array containing the data and uncertainties
    #           N = the number of participants
    #           d = matrix to store the differences
    # Returns:  d = matrix containing all the differences
    #           w_LCS = array showing which labs have been excluded in LCS
    #           t = size of the largest coherent subset minus one, used as subset size in BMA
    
    # No cut with and without OO, MP
    (d[0:], u_d[0:]) = NoCut(y,N,True,True)
    (d[1:], u_d[1:]) = NoCut(y,N,True,False)
    (d[2:], u_d[2:]) = NoCut(y,N,False,True)
    (d[3:], u_d[3:]) = NoCut(y,N,False,False)

    # Cut with and without OO, MP
    (d[4:], u_d[4:]) = Cut(y,N,True,True)
    (d[5:], u_d[5:]) = Cut(y,N,True,False)
    (d[6:], u_d[6:]) = Cut(y,N,False,True)
    (d[7:], u_d[7:]) = Cut(y,N,False,False)

#     # LChS, BMA, LCS
#     lchs = LChS(y,N)
    t = 0

#     d[8:] = BMA(y,N,t)
    (d[8:], u_d[8:]) = LCS(y,N)

    return(d,t,u_d)

In [5]:
def success(d,u_d,fail):
    # Determine if each method fails, and if it does update the matrix fail. If the biased lab fails, update the array 
    # bias_fail.
    # Takes in: d = matrix containing all the differences
    #           w_LCS = array showing which labs have been excluded in LCS
    #           fail = array in which to store the number of labs which fail
    # Returns: fail = array containing the number of labs which failed using each method.
    
    temp = abs(d) > 1.96*u_d    
    fail[:,] += temp

    return(fail)

In [6]:
def simulate_data(T,N,f,g,df):
    # Generate datasets, and tests them to see how many labs fail using each method
    # Takes in: T = number of trials
    #           N = number of participants
    #           f = uncertainty of biased lab
    #           g = size of bias
    #           d = degrees of freedom used to generate biased labs
    # Returns:  fail_rate = matrix containing the failure rates of each lab
    #           t_ave_final = average subset size fed into BMA
    
    
    # Create array in which to store the failure rates, and differences
    fail = np.zeros((9,N))
    d = la.uarray(np.zeros((9,N)),np.zeros((9,N)))
    u_d = la.uarray(np.zeros((9,N)),np.zeros((9,N)))
    
    # Initialise stop (to break out of while loop), t_ave
    stop = T * 1.
    t_ave = 0

    while stop > 0:
        # Create participants
        [y, n_biased] = participants(N,f,g,df)       
        
        # Run participant set through all the different methods
        (d,t,u_d) = run_methods(y,N,d,u_d)
        
        # Update t_ave
        t_ave += t
        
        # Calculate the failure rates and update fail
        fail = success(d,u_d,fail)
        
        # Update stop, t_ave
        stop -= 1
    
    # Calculate fail rates by dividing the number of labs which failed by the number of trials
    fail_rate = fail/T
    t_ave_final = t_ave/T
    
    return(fail_rate, t_ave_final, n_biased)

In [23]:
N = 12
T = 1
df = 4
m = 5

f_all = [0.25]
g_all = [0.09375]

for f in f_all:
    for g in g_all:

        # Initliase spreadsheet to save results
        wb = xlwt.Workbook()
        ws = wb.add_sheet('Fail rate')

        # Generate failure rates
        fail_rates, t, n_biased = simulate_data(T,N,f,g,df)
        
        if n_biased == 1:
            ws.write(1,0,'Biased')
        elif n_biased == 2:
            ws.write(2,0,'Biased')
            ws.write(3,0,'Biased')
     
        ws.write(0,1,'No cut (OO,MP)')
        ws.write(0,2,'No cut (OO)')
        ws.write(0,3,'No cut (MP)')
        ws.write(0,4,'No cut')
        ws.write(0,5,'Cut (OO,MP)')
        ws.write(0,6,'Cut (OO)')
        ws.write(0,7,'Cut (MP)')
        ws.write(0,8,'Cut')
        ws.write(0,9,'LCS')
        
        ws.write(0,11,'Version ' + str(code_version))

        # Write results to spreadsheet
        for i in range(0,N):
            for j in range(0,9):
                ws.write(i+1,j+1,fail_rates[j,i])

        wb.save('two_bias+-,f='+str(f)+',g='+str(g)+'.xls')